這份 Notebook 示範如何用 Prompt Chaining 來串接外部工具，包括 Google 搜尋、計算機和股價 API

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2000, format_type="text"):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens, "response_format": { "type": format_type } }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

## 案例一: Google 搜尋

OpenAI gpt-3.5-turbo 模型的 cut-off date 是 2021/9，最新 GPT-4-0125-preview 是 2023/12。因此沒有最新資料。

我們可以透過搜尋提供參考資料，讓模型基於參考資料來回答，減少亂掰的可能性。

In [4]:
# 這是非官方的 google 爬蟲
!pip install googlesearch-python

# 若要用官方 JSON API https://developers.google.com/custom-search/v1/overview?hl=zh-tw (有 API key 需付費但有免費額度)

  Preparing metadata (setup.py) ... done
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4209 sha256=e03b61a2af36cb269622206aa1ad7e4a811e1084d85e1caf2edfb29ad1dbbd41
  Stored in directory: /root/.cache/pip/wheels/98/24/e9/6c225502948c629b01cc895f86406819281ef0da385f3eb669
Successfully built googlesearch-python


In [5]:
from googlesearch import search

In [6]:
def google_search(keyword):
  content = ""
  for item in search(keyword, advanced=True, num_results=5, lang='zh-tw'):
    content += f"Title: {item.title}\n Description: {item.description}\n\n"
  return content

In [7]:
google_search("ihower")

'Title: ihower { blogging } – \u200d  \n Description: 先分享一個時代情懷，這是xkcd 在2014 的梗圖，當時是想表達在CS領域中，很難跟一般人解釋簡單和幾乎不可能的任務。沒想到十年後的今天，這個影像解讀的任務也已經變成簡單\xa0...\n\nTitle: ihower { blogging } – \u200d  \n Description: 先分享一個時代情懷，這是xkcd 在2014 的梗圖，當時是想表達在CS領域中，很難跟一般人解釋簡單和幾乎不可能的任務。沒想到十年後的今天，這個影像解讀的任務也已經變成簡單\xa0...\n\nTitle: Generative AI Engineer (LLM) 專業教育訓練\n Description: ihower 的Generative AI Engineer 專業教育訓練| 愛好資訊科技.\n\nTitle: Generative AI Engineer (LLM) 專業教育訓練\n Description: ihower 的Generative AI Engineer 專業教育訓練| 愛好資訊科技.\n\nTitle: Generative AI Engineer (LLM) 專業教育訓練\n Description: ihower 的Generative AI Engineer 專業教育訓練| 愛好資訊科技.\n\n'

In [8]:
user_query = "請問 2024 年 Nvidia 的股價表現如何?"

prompt = f"""
你是 Google 搜尋引擎，請根據以下用戶問題，擷取出和問題相關的重要關鍵字:
用戶問題: ```{user_query}```
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
keywords = response
pp(response)

'重要關鍵字: 2024年, Nvidia, 股價表現'


💡 AI 回覆有時候會多 label，例如上面的 "重要關鍵字"
這時候可以改一下 prompt 的結尾，加上引導文字，讓 AI 繼續接龍下去。

## 加上引導文字

In [9]:
prompt = f"""
你是 Google 搜尋引擎，請根據以下用戶問題，擷取出和問題相關的重要關鍵字:
用戶問題: ```{user_query}```
關鍵字:
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
keywords = response
pp(response)

'2024年、Nvidia、股價、表現。'


In [10]:
search_result = google_search(keywords)
search_result

'Title: 輝達概念股有哪些？未來前景如何？2024年 ...\n Description: 2 天前 — 踏入2024年之後，輝達在股票市場的表現愈發成爲市場關注的焦點。截至2024年3月1日，輝達的YTD股價漲幅已經高達70.82%。特別是在2月22日公佈亮眼的財報\xa0...\n\nTitle: 2024/2/23輝達股價大漲16%，美股多頭確立與債券市場展望\n Description: 2024年2月23日 — 在這則影片中，主持人回顧了2024年2月23日美股的表現，特別聚焦於輝達（NVIDIA）財報公佈後的股價大漲以及美國股市多頭確立的情況。\n\nTitle: NVDA 股票可能在2024 年分割：更新- moomoo 社區\n Description: 2024年1月14日 — Nvidia 擁有令人印象深刻的增長歷史，但對其人工智能驅動的成果的興奮使該股在2023 年上漲239％。考慮過去10 年，它的表現更明顯。收入飆升1,480%，使\xa0...\n\nTitle: 美股史上最快NVIDIA只花8個月達標2兆美元市值\n Description: 2024年2月25日 — 受惠AI趨勢，2023年NVIDIA股價累計大漲238.87%，2024年至今續漲59.16 ... 股價表現勢必不俗。 (圖片來源：NVIDIA). ＊編者按：本文僅供參考之用，並不\xa0...\n\nTitle: NVIDIA輝達2024年第四季度財報有什麼重點？業務表現如何？\n Description: 2024年2月25日 — NVIDIA輝達在2024年第四季度的財報中呈現了哪些重點？該公司是一家以加速計算為核心的企業，致力於解決各種複雜的計算問題。如今，NVIDIA已經發展成為\xa0...\n\nTitle: Nvidia英偉達的財報將透露2024年AI市場需求 - Convo Media\n Description: 英偉達的股價在2023年飆升了237%，遠遠超過標準普爾500指數中的任何其他成員。其市值目前為1.2兆美元，遠高於Meta或特斯拉。財報電話會議上有任何跡象表明\xa0...\n\nTitle: Nvidia股價表現驚人\n Description: 2024年2月24日 — 2024年到目前為止，Nvidia的股價一直持續上升，

In [11]:
prompt = f"""

請基於以下最新事實回答:

{search_result}

用戶問題: {user_query}
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
print(response)

根據最新的資訊，2024年Nvidia的股價表現非常亮眼。截至2024年3月1日，Nvidia的YTD股價漲幅已達到70.82％，並且在2月23日股價更是大漲了16％。這顯示Nvidia在2024年的表現非常強勁，市場對其前景持樂觀態度。另外，Nvidia可能在2024年進行股票分割，這也反映了市場對該公司的信心和期待。總的來說，2024年Nvidia的股價表現令人印象深刻，顯示該公司在股票市場上的強勁表現。


## ✅ 串接工具的基本模式

1. 從用戶問題中，用 prompt1 來提取出 外部工具的參數
2. 執行外部工具，拿到結果
3. 用 (prompt2 + 結果) 轉成自然語言回給用戶

## 案例二: 計算機

我們知道 LLM 對於複雜的數學運算不是很在行。
就跟人一樣，心算也不在行，用計算機最快!

Step 1: 提取參數

In [ ]:
question = "請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234"

# prompt 參考自 langchain LLMMathChain
prompt = f"""
Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
Expression: ${{single line mathematical expression that solves the problem}}

Begin.

Question: What is 37593 * 67?
Expression: 37593 * 67

Question: 37593^(1/5)
Expression: 37593**(1/5)

Question: {question}
Expression:
"""

In [ ]:
prompt

"\nTranslate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${Question with math problem.}\nExpression: ${single line mathematical expression that solves the problem}\n\nBegin.\n\nQuestion: What is 37593 * 67?\nExpression: 37593 * 67\n\nQuestion: 37593^(1/5)\nExpression: 37593**(1/5)\n\nQuestion: 請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234\nExpression:\n"

In [ ]:
messages = [{"role": "user", "content": prompt }]

response = get_completion(messages, model="gpt-4")
pp(response)

'((64 * 2 - 8) / 100) ** 1.234'


Step 2: 執行工具

In [ ]:
import numexpr

In [ ]:
answer = numexpr.evaluate(response)

In [ ]:
str(answer)

'1.2523036823884173'

Step 3: 回給用戶

執行工具後的回傳，你可以選擇

1. 直接回給用戶 或是 轉成某種固定格式回給客戶，就不用再一次呼叫 LLM 了
3. 再丟給 LLM 變成自然語言再回給客戶


In [ ]:
prompt = f"""
這是用戶問題: {question}

這其中的數學問題: {response} 和答案: {answer}
請回答:
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
pp(response)

'答案是 1.2523036823884173。'


## 案例三: 串接股價 API

這次我們要求 json 格式

### Step 1: 從用戶問題中，用 prompt1 來提取出 外部工具的參數

In [ ]:
query = "請問113年的3月5號股價表現如何?"

messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

1. 無需回答問題，請從用戶問題中，擷取出日期和台灣股票代號
2. 如果日期是民國年份，請加上 1911 轉成西元年份
3. 如果用戶沒有提供日期和公司名稱或股票代號，請回傳 {{ "error": "string" // 錯誤訊息 }}
4. 請回覆 JSON 格式，例如
{{
  "date": "20231015", // yyyymmdd
  "stock_code": "0050" // 台灣的股票代號
}}"""
}]

result = get_completion(messages, model="gpt-3.5-turbo-1106", format_type="json_object" )
print(result)

data = json.loads(result)

{
  "error": "請提供日期和股票代號"
}


In [ ]:
query = "請問 113年的3月5號的台積電，股價表現如何?"

messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

1. 無需回答問題，請從用戶問題中，擷取出日期和台灣股票代號
2. 如果日期是民國年份，請加上 1911 轉成西元年份
3. 如果用戶沒有提供日期和公司名稱或股票代號，請回傳 {{ "error": "string" // 錯誤訊息 }}
4. 請回覆 JSON 格式，例如
{{
  "date": "20231015", // yyyymmdd
  "stock_code": "0050" // 台灣的股票代號
}}"""
}]

result = get_completion(messages, model="gpt-3.5-turbo-1106", format_type="json_object" )
print(result)

data = json.loads(result)

{
  "date": "20240305",
  "stock_code": "2330"
}


### Step 2: 執行工具，拿到結果

API 參考自 https://medium.com/%E5%B7%A5%E7%A8%8B%E9%9A%A8%E5%AF%AB%E7%AD%86%E8%A8%98/5%E7%A8%AE%E6%8A%93%E5%8F%96%E5%8F%B0%E8%82%A1%E6%AD%B7%E5%8F%B2%E8%82%A1%E5%83%B9%E7%9A%84%E6%96%B9%E6%B3%95-766bf2ed9d6

In [ ]:
date = data["date"]
stock_code = data["stock_code"]

html = requests.get('https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=%s&stockNo=%s' % (date,stock_code))
context = json.loads(html.text)
context

{'stat': 'OK',
 'date': '20240305',
 'title': '113年03月 2330 台積電           各日成交資訊',
 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'],
 'data': [['113/03/01',
   '24,167,721',
   '16,699,995,060',
   '697.00',
   '697.00',
   '688.00',
   '689.00',
   '-1.00',
   '26,282'],
  ['113/03/04',
   '97,210,112',
   '69,868,348,694',
   '714.00',
   '725.00',
   '711.00',
   '725.00',
   '+36.00',
   '125,799'],
  ['113/03/05',
   '73,299,411',
   '53,751,887,376',
   '735.00',
   '738.00',
   '728.00',
   '730.00',
   '+5.00',
   '69,851']],
 'notes': ['符號說明:+/-/X表示漲/跌/不比價',
  '當日統計資訊含一般、零股、盤後定價、鉅額交易，不含拍賣、標購。',
  'ETF證券代號第六碼為K、M、S、C者，表示該ETF以外幣交易。',
  '權證證券代號可重複使用，權證顯示之名稱係目前存續權證之簡稱。'],
 'total': 3}

### Step 3: 用 (prompt2 + 結果) 轉成自然語言回給用戶

In [ ]:
messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

context: {context}
"""}
]

result = get_completion(messages, model="gpt-3.5-turbo-1106")
print(result)

113年3月5號的台積電股價表現如下：
- 成交股數：73,299,411
- 成交金額：53,751,887,376
- 開盤價：735.00
- 最高價：738.00
- 最低價：728.00
- 收盤價：730.00
- 漲跌價差：+5.00
- 成交筆數：69,851

從數據來看，台積電在113年3月5號的股價有上漲，收盤價為730.00。


## ✅ 串接工具的基本模式

1. 從用戶問題中，用 prompt1 來提取出 外部工具的參數
2. 執行外部工具，拿到結果
3. 用 (prompt2 + 結果) 轉成自然語言回給用戶